In [1]:
import pandas as pd
import numpy as np
import json
import pickle
import sklearn.metrics
from sklearn import preprocessing
from scipy.sparse import coo_matrix

In [2]:
with open("utility_matrix.pkl", "rb") as f:
    utility = pickle.load(f)

In [3]:
U_csc = utility.tocsc()

In [4]:
U_csr = utility.tocsr()

In [5]:
# Get items rated by "our" user
our_user = U_csr.getrow(4)
rows, cited_item_idxs = our_user.nonzero()

In [6]:
# for each of these items we need to calculate similar items
pred_rtgs = []

k = 30

for idx in cited_item_idxs:
    curr_item = U_csc.getcol(idx).T
    sims = sklearn.metrics.pairwise.cosine_similarity(curr_item, U_csc.T)
    sims = np.squeeze(sims)
    asort = sims.argsort()
    k_most_similar_idx = asort[::-1][1:k+1]
    k_sims = sims[k_most_similar_idx]
    cols_k_similar = np.squeeze(np.array([U_csr.getcol(r).todense() for r in k_most_similar_idx]))
    pred_from_one_item = cols_k_similar.mean(axis=0)
    pred_rtgs.append(pred_from_one_item)
    
predicted_ratings = np.array(pred_rtgs).mean(axis=0)

In [8]:
predicted_ratings.mean()

1.613761000781209e-05

In [8]:
asort = predicted_ratings.argsort()
asort = asort[::-1] # to descending order

b = 5
predicted_idxs = asort[1:b+1]

In [10]:
with open("items_encoder.pkl", "rb") as f:
    items_encoder = pickle.load(f)
    
with open("users_encoder.pkl", "rb") as f:
    users_encoder = pickle.load(f)

In [13]:
with open("id2title.pkl", "rb") as f:
    id2title = pickle.load(f)
    
with open("id2name.pkl", "rb") as f:
    id2name = pickle.load(f)
    
with open("name2id.pkl", "rb") as f:
    name2id = pickle.load(f)
    
with open("title2id.pkl", "rb") as f:
    title2id = pickle.load(f)

In [11]:
predicted_idxs
predicted_ids = items_encoder.inverse_transform(predicted_idxs)

In [12]:
predicted_ids

array([2011710469, 1972620114, 1530404001, 2063604850, 2160460946])

In [14]:
for i in predicted_ids:
    print(id2title[i])

Multi-symplectic methods for generalized Schrödinger equations
Backward error analysis for multi-symplectic integration methods
Simulating Hamiltonian Dynamics
On Symplectic and Multisymplectic Schemes for the KdV Equation
Linear PDEs and Numerical Methods That Preserve a Multisymplectic Conservation Law
